In [ ]:
import pandas as pd
import os
import plotly.express as px

FILE_PATH = os.path.join("data", "2021-11-03_4M1QM_suppliers.txt")

ENCODING_FIXES = {
    "NÃ¼rnberg": "Nürnberg",
    "Ã¼": "ü",
    "Ã¤": "ä",
    "Ã¶": "ö",
    "ÃŸ": "ß",
    "Ã©": "é",
    "Ã€": "À",
    "Ã©": "é",
    "Ãˆ": "È",
    "Ã´": "ô",
    "ÃŠ": "Ê",
    "â‚¬": "€",
    "â€œ": "“",
    "â€\x9d": "”",
    "â€“": "-",
    "â€”": "—",
    "â€™": "'",
}

def fix_encoding(text):
    """Manually replace incorrectly decoded characters."""
    for wrong, correct in ENCODING_FIXES.items():
        text = text.replace(wrong, correct)
    return text

def parse_suppliers_file(file_path):
    """Reads and parses the supplier file with encoding fixes."""
    
    suppliers = []
    
    with open(file_path, "r", encoding="utf-8", errors="replace") as file:
        lines = (line.strip() for line in file if line.strip())
        
        while True:
            try:
                supplier_id = next(lines)
                next(lines)
                
                services = []
                while True:
                    line = next(lines)
                    if line == "------":
                        break
                    services.append(fix_encoding(line))

                address = fix_encoding(next(lines))

                # join set to remove duplicates
                suppliers.append({
                    "Supplier ID": supplier_id,
                    "Services": ", ".join(set(services)),
                    "Address": address
                })
                
            except StopIteration:
                break

    return pd.DataFrame(suppliers)

df_suppliers_from_text = parse_suppliers_file(FILE_PATH)

df_suppliers_from_text

,Supplier ID,Services,Address
0,6XF0E43VOTV0V,"Laser Cutting, CAD, Grinding, Sheet Metal Bending","Omprtzgjab Street 118, Nürnberg"
1,6XF0E43VOTV0V,"Laser Cutting, CAD, Grinding, Sheet Metal Bending","Omprtzgjab Street 118, Nürnberg"
2,W10VLTND6ZX,"Wire Cutting, Delicate Turning Parts, Gears, S...","Seattle, Plfdmlwlxb Street 58"
3,EYV0B2QG,"Wire Cutting, Injection Molding, Delicate Turn...","Omprtzgjab Street 118, Nürnberg"
4,I7FEPG6PA9,"Timber, Gears, Delicate Parts, Grinding, Anodi...","Omprtzgjab Street 118, Nürnberg"
5,9RFK3W,"Gears, Delicate Parts, Large Turning Parts, Gr...","Plfdmlwlxb Street 58, Seattle"
6,9RFK3W,"Gears, Delicate Parts, Large Turning Parts, Gr...","Plfdmlwlxb Street 58, Seattle"
7,KJRZYJA,"Injection Molding, Delicate Parts, Plastic","Omprtzgjab Street 118, Nürnberg"
8,PWC8OX45M86EQNO,"Grinding, Profiles, Laser Cutting, Sheet Metal...","Nürnberg, Omprtzgjab Street 118"
9,9DPQH5A69MFVK,"Timber, Delicate Turning Parts, Large Turning ...","Nrcnmtpqoc Street 58, Stuttgart"


In [76]:
import sqlite3
import pandas as pd

DB_FILE = "orders_db.sqlite"
conn = sqlite3.connect(DB_FILE)
cursor = conn.cursor()

cursor.executescript("""
drop table stg_table;
drop table Orders;
drop table Suppliers;
drop table Customers;

create table Suppliers(
    id integer primary key,
    name varchar(100));

create table Customers(
    id integer primary key,
    name varchar(100));

create table Orders(
    id integer primary key,
	supplier_id int references Suppliers(id),
	customer_id int references Customers(id),
    order_value numeric,
    requested_delivery_date date,
    actual_delivery_date date,
	customer_rating numeric);

create table stg_table (
	supplier_nm varchar(100),
	customer_nm varchar(100),
	order_value numeric,
    requested_delivery_date date,
    actual_delivery_date date,
	customer_rating numeric);
""")

conn.commit()

csv_file = "data/2021-11-03_HDCE3_orders_dataset.csv"
df = pd.read_csv(csv_file, delimiter=";")

df.to_sql("stg_table", conn, if_exists="append", index=False)

cursor.execute("insert into Suppliers (name) select distinct(supplier_nm) from stg_table")

cursor.execute("insert into Customers (name) select distinct(customer_nm) from stg_table")

cursor.execute("""
insert into Orders (supplier_id, customer_id, order_value, requested_delivery_date, actual_delivery_date, customer_rating)
select su.id, c.id, s.order_value, s.requested_delivery_date, s.actual_delivery_date, s.customer_rating
from stg_table as s
join Suppliers as su
on su.name = s.supplier_nm
join Customers as c
on c.name = s.customer_nm;
""")

conn.commit()

df_orders = pd.read_sql("SELECT * FROM Orders", conn)
df_customers = pd.read_sql("SELECT * FROM Customers", conn)
df_suppliers = pd.read_sql("SELECT * FROM Suppliers", conn)

conn.close()


In [77]:
### Small data cleansing
df_suppliers_from_text.drop_duplicates(inplace=True)
df_orders['customer_rating'].fillna(0, inplace=True)

### New features
## delays
df_orders['requested_delivery_date'] = pd.to_datetime(df_orders['requested_delivery_date'], errors='coerce')
df_orders['actual_delivery_date'] = pd.to_datetime(df_orders['actual_delivery_date'], errors='coerce')
df_orders["order_delay_days"] = (df_orders["actual_delivery_date"] - df_orders["requested_delivery_date"]).dt.days


## Orders Frequency
# Orders per customer
customer_order_counts = df_orders["customer_id"].value_counts().reset_index()
customer_order_counts.columns = ["customer_id", "customer_order_frequency"]

# Orders per supplier
supplier_order_counts = df_orders["supplier_id"].value_counts().reset_index()
supplier_order_counts.columns = ["supplier_id", "supplier_order_frequency"]

## Number of Suppliers per Customer
customer_supplier_counts = df_orders.groupby("customer_id")["supplier_id"].nunique().reset_index()
customer_supplier_counts.columns = ["customer_id", "num_suppliers"]

## Number of Customers per Supplier
supplier_customer_counts = df_orders.groupby("supplier_id")["customer_id"].nunique().reset_index()
supplier_customer_counts.columns = ["supplier_id", "num_customers"]

## Order Value for Customers
customer_order_values = df_orders.groupby("customer_id")["order_value"].sum().reset_index()
customer_order_values.columns = ["customer_id", "total_order_value_customer"]

## Order Value for Suppliers
supplier_order_values = df_orders.groupby("supplier_id")["order_value"].sum().reset_index()
supplier_order_values.columns = ["supplier_id", "total_order_value_supplier"]

/tmp/ipykernel_7704/1453546454.py:3: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [78]:
## Cleansing
# drop duplicates
df_suppliers_from_text.drop_duplicates(inplace=True)

# rename column
df_suppliers_from_text.rename(columns={"Supplier ID": "supplier_id"}, inplace=True)

# one hot encode the services
df_suppliers_from_text = df_suppliers_from_text.assign(
    Services=df_suppliers_from_text['Services'].str.split(', ')
).explode('Services')

df_suppliers_from_text = pd.get_dummies(df_suppliers_from_text, columns=['Services'])

In [81]:
# select top 10 customers by total order value
top_customers = customer_order_values.nlargest(10, "total_order_value_customer")

fig = px.bar(
    top_customers,
    x="customer_id",
    y="total_order_value_customer",
    title="Top 10 Most Valuable Customers",
    labels={"customer_id": "Customer ID", "total_order_value_customer": "Total Order Value"},
    text_auto=True
)

In [82]:
# best suppliers analysis

# total sum delays per supplier
supplier_delays = df_orders.groupby("supplier_id")["order_delay_days"].sum().reset_index()
supplier_delays.columns = ["supplier_id", "total_delay_days"]
supplier_metrics = supplier_delays.merge(supplier_customer_counts, on="supplier_id").merge(supplier_order_values, on="supplier_id")

# rank suppliers based on criteria
supplier_metrics["rank_delay"] = supplier_metrics["total_delay_days"].rank(ascending=True, method="min")
supplier_metrics["rank_customers"] = supplier_metrics["num_customers"].rank(ascending=False, method="min")
supplier_metrics["rank_order_value"] = supplier_metrics["total_order_value_supplier"].rank(ascending=False, method="min")

supplier_metrics["overall_score"] = supplier_metrics["rank_delay"] + supplier_metrics["rank_customers"] + supplier_metrics["rank_order_value"]
best_suppliers = supplier_metrics.sort_values(by="overall_score").reset_index(drop=True)

In [84]:
# top 10 best-performing suppliers based on overall score
top_suppliers = best_suppliers.head(10)

fig = px.bar(
    top_suppliers,
    x="supplier_id",
    y=["total_delay_days", "num_customers", "total_order_value_supplier"],
    title="Top 10 Best Performing Suppliers",
    labels={"supplier_id": "Supplier ID", "value": "Metric Value"},
    barmode="group",
)

fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
print(best_suppliers[:10])

,supplier_id,total_delay_days,num_customers,total_order_value_supplier,rank_delay,rank_customers,rank_order_value,overall_score
0,14,59,32,257093.93,1.0,6.0,5.0,12.0
1,9,62,31,232940.79,2.0,7.0,11.0,20.0
2,10,79,33,263356.84,14.0,3.0,3.0,20.0
3,1,82,34,314455.79,17.0,2.0,2.0,21.0
4,8,77,33,255653.06,13.0,3.0,6.0,22.0
5,6,76,33,236188.76,12.0,3.0,9.0,24.0
6,19,72,30,263324.30,10.0,12.0,4.0,26.0
7,12,87,37,243054.39,18.0,1.0,8.0,27.0
8,2,95,31,324640.80,20.0,7.0,1.0,28.0
9,11,63,31,202967.74,4.0,7.0,20.0,31.0
